In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Load dataset
file_path = "../Dataset/EURUSD/EURUSD_M30_feature_v.2.csv"
data = pd.read_csv(file_path)

# Add a label column with the value of the next candle's Close
data['Label'] = data['Close'].shift(-1)
data = data.dropna()

# Features and labels
features = ['Open', 'High', 'Low', 'Close', 'SMA200', 'SMA100', 'SMA50', 'SMA13', 'RSI14']
label = 'Label'

# Train-test split with time-series considerations
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])
data[label] = scaler.fit_transform(data[[label]])

train_size = int(len(data) * 0.8)
train_data, test_data = data[:train_size], data[train_size:]

# PyTorch Dataset
class ForexDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.features = data[features].values
        self.labels = data[label].values
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.features[idx:idx + self.sequence_length]
        y = self.labels[idx + self.sequence_length - 1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

sequence_length = 30
train_dataset = ForexDataset(train_data, sequence_length)
test_dataset = ForexDataset(test_data, sequence_length)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

# Neural Network Architecture
class ForexModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(ForexModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_sizes[2], output_size),
            nn.Tanh()  # Constrains the output to [-1, 1]
        )

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.fc(x[:, -1, :])  # Take the last time step's output
        return x

input_size = len(features)
hidden_sizes = [128, 64, 32]  # Descending sizes for LSTM layers
output_size = 1

model = ForexModel(input_size, hidden_sizes, output_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

epochs = 20
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "./Model/3layer_lstm.v.6.model")


Epoch 1/20, Loss: 0.1251
Epoch 2/20, Loss: 0.0100
Epoch 3/20, Loss: 0.0054
Epoch 4/20, Loss: 0.0023
Epoch 5/20, Loss: 0.0008
Epoch 6/20, Loss: 0.0005
Epoch 7/20, Loss: 0.0006
Epoch 8/20, Loss: 0.0006
Epoch 9/20, Loss: 0.0006
Epoch 10/20, Loss: 0.0006
Epoch 11/20, Loss: 0.0005
Epoch 12/20, Loss: 0.0005
Epoch 13/20, Loss: 0.0005
Epoch 14/20, Loss: 0.0005
Epoch 15/20, Loss: 0.0004
Epoch 16/20, Loss: 0.0004
Epoch 17/20, Loss: 0.0004
Epoch 18/20, Loss: 0.0003
Epoch 19/20, Loss: 0.0003
Epoch 20/20, Loss: 0.0002


: 